In [ ]:
import pandas as pd
import glob
import numpy
import math
from math import pi
import calendar
from pylab import *
from collections import defaultdict

In [ ]:
def date_period(time1,time2):
    a1 = time1
    a2 = time2
    b1 = a1.split('-')
    b2 = a2.split('-')
    c1=datetime.date(int(b1[0]), int(b1[1]), int(b1[2]))   #开始的日期de
    c2=datetime.date(int(b2[0]), int(b2[1]), int(b2[2]))  #  结束的日期
    result=(c2-c1).days
    return result

In [ ]:
def Wang_engle(T,Tbase,Topt,Tcei):
    # '''
    # Wang and Engle 1998, Agircultual systems.
    # beta function: the effect of temperature on development rate
    # '''
    if T<=Tbase or T>=42:
        return 0
    else:
        alpha=math.log(2, )/(math.log((Tcei-Tbase)/(Topt-Tbase)))
        dr=(2*((T-Tbase)**alpha)*(Topt-Tbase)**alpha-(T-Tbase)**(2*alpha))/((Topt-Tbase)**(2*alpha))
        return dr*(T-Tbase)

In [ ]:
def photo_period_based_on_yin(mu,zeta,ep,dl):
    # '''
    # Yin 1997 beta function
    # the effect of photoperiod on development rate
    # '''
    return math.exp(mu)*(dl)**zeta*(24-dl)**ep

In [ ]:
data=pd.read_csv('D:/workspace/rice_crop_model/data/Rice_phen_ob.csv',encoding='gbk')

<!-- -----------------Photoperiod sensitive phase-----------------
calculate daily thermal, daily photoeffect, daily photothermal, 
cumumlate thermal, cumumlate photothermal form reviving to heading -->

In [ ]:
f2=defaultdict(list)
f1=defaultdict(list)
for i in range(data.shape[0]):
    stardate=pd.Timestamp(data["reviving data"][i]).date().strftime('%Y-%m-%d')
    enddate=pd.Timestamp(data['heading data'][i]).date().strftime('%Y-%m-%d')
    n=date_period(stardate,enddate)
    lon=data["lon"][i]
    lat=data["lat"][i]
    ID=data["station ID"][i]
    year=int(stardate[:4])
    
    metedata=pd.read_table("D:/workspace/rice_crop_model/data/Meteo(48 sta)/"+str(ID)+".txt",encoding='gbk',sep=' * ',engine='python')
    mete=metedata.drop(index=0,axis=1) #去掉第一行
    mete['ymd']=mete['YY']+'-'+mete['mm']+'-'+mete['dd']
    mete.index=pd.DatetimeIndex(mete['ymd'])
    
    cumumlate_photothermal=0
    cumumlate_thermal=0
    
    for j in range (n+1):
        stardate1=(pd.Timestamp(data["reviving data"][i])+pd.Timedelta(days=j)).date().strftime('%Y-%m-%d') 
        metedf=mete[mete.index==stardate1]
        Temp=float(metedf['TemAver'])
        month=int(stardate1[-5:-3])
        day=int(stardate1[-2:])
        parameter = Sun()
        daylength=parameter.dayCivilTwilightLength(year, month, day, lon, lat)
        photoeffect= photo_period_based_on_yin(-15.46,2.06,2.48,daylength)
        daily_thermal=Wang_engle(Temp,8,30,42)
        daily_photothermal=daily_thermal*photoeffect
        cumumlate_thermal+=daily_thermal
        cumumlate_photothermal+=daily_photothermal
        f1["Station_ID"].append(ID)
        f1["year"].append(year)
        f1["data_PSP"].append(stardate1)
        f1["photoeffect"].append(photoeffect)
        f1["daily_thermal_PSP"].append(daily_thermal)
        f1["daily_photothermal"].append(daily_photothermal)
          
    
    f2["Station_ID"].append(ID)
    f2["year"].append(year)
    f2["cumumlate_thermal_PSP"].append(cumumlate_thermal)
    f2["cumumlate_photothermal"].append(cumumlate_photothermal)
F1=pd.DataFrame(f1)
F1.to_csv("D:/workspace/rice_crop_model/data/daily_thermal_PSP.csv",index=False,encoding='gbk')             
F2=pd.DataFrame(f2)
F2.to_csv("D:/workspace/rice_crop_model/data/cumumlate_thermal_PSP.csv",index=False,encoding='gbk')

<!-- ----------Reproductive growth phase-----------------
calculate daily thermal, cumumlate thermal form heading to maturity -->

In [ ]:
f2=defaultdict(list)
f1=defaultdict(list)
for i in range(data.shape[0]):
    stardate=pd.Timestamp(data["heading data"][i]).date().strftime('%Y-%m-%d')
    enddate=pd.Timestamp(data['maturity data'][i]).date().strftime('%Y-%m-%d')
    n=date_period(stardate,enddate)
    lon=data["lon"][i]
    lat=data["lat"][i]
    ID=data["station ID"][i]
    year=int(stardate[:4])
    
    metedata=pd.read_table("D:/workspace/rice_crop_model/data/Meteo(48 sta)/"+str(ID)+".txt",encoding='gbk',sep=' * ',engine='python')
    mete=metedata.drop(index=0,axis=1) #去掉第一行
    mete['ymd']=mete['YY']+'-'+mete['mm']+'-'+mete['dd']
    mete.index=pd.DatetimeIndex(mete['ymd'])
    
    cumumlate_thermal=0
    
    for j in range (n+1):
        stardate1=(pd.Timestamp(data["heading data"][i])+pd.Timedelta(days=j+1)).date().strftime('%Y-%m-%d') 
        metedf=mete[mete.index==stardate1]
        Temp=float(metedf['TemAver'])
        month=int(stardate1[-5:-3])
        day=int(stardate1[-2:])
        parameter = Sun()
        daylength=parameter.dayCivilTwilightLength(year, month, day, lon, lat)
        daily_thermal=Wang_engle(Temp,8,30,42)
        cumumlate_thermal+=daily_thermal
        f1["Station_ID"].append(ID)
        f1["year"].append(year)
        f1["data_RGP"].append(stardate1)
        f1["daily_thermal_RGP"].append(daily_thermal)
          
    f2["Station_ID"].append(ID)
    f2["year"].append(year)
    f2["cumumlate_thermal_RGP"].append(cumumlate_thermal)

F1=pd.DataFrame(f1)
F1.to_csv("D:/workspace/rice_crop_model/data/daily_thermal_RGP.csv",index=False,encoding='gbk')             
F2=pd.DataFrame(f2)
F2.to_csv("D:/workspace/rice_crop_model/data/cumumlate_thermal_RGP.csv",index=False,encoding='gbk')
